Most of this was copied from translate 2020

In [ ]:
import pandas as pd
import numpy as np
import re
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [ ]:
def read_excel_sheets(xls_path):
    """Read all sheets of an Excel workbook and return a single DataFrame"""
    print(f'Loading {xls_path} into pandas')
    xl = pd.ExcelFile(xls_path)
    clslist = []
    sheetnames =[]
    columns = None
    for idx, name in enumerate(xl.sheet_names):
        print(f'Reading sheet #{idx}: {name}')
        sheet = xl.parse(name)
        # Assume index of existing data frame when appended
        clslist.append(sheet)
        sheetnames.append(name)
    return clslist, sheetnames
cls2021, names = read_excel_sheets("C:/Users/parke/OneDrive/Documents/GitHub/Attollo/MasterSheet CSVS/Master Contact Sheet 2019-2020 Junior Scholars.xlsx")
for i in range(len(names)):
    names[i] = names[i].rstrip()

## Clean columns

In [ ]:
def cleancolumns(df):
    #removes unnamed
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    if 'Staff Initials' in df.columns:
        del df['Staff Initials']
    if 'Staff initials' in df.columns:
        del df['Staff initials']
    if 'Staff Initial' in df.columns:
        del df['Staff Initial']
    if 'Last Update' in df.columns:
        del df['Last Update']
    if 'Last Updated' in df.columns:
        del df['Last Updated']
    return df
for i in range(7):
    cls2021[i]=cleancolumns(cls2021[i])
    
def pcol(df):
    print(df.columns.size, df.columns)
for i in range(7):
    pcol(cls2021[i])

## Make all into same df

In [ ]:
allcols = cls2021[2].columns

#add columns that dont exist
count =1
for df in cls2021[:6]: 
    print(names[count-1], '-----')
    for c in allcols:
        if c not in df.columns:
            df[c]=np.nan
    df=df.reindex(allcols, axis ="columns")
    count+=1

#concatenate into one dataframe
schools = names[:6]
alldf= pd.DataFrame()
for i in range(6):
    cls2021[i]["School"] = schools[i]
alldf = pd.concat(cls2021[:6], ignore_index =True)
alldf.columns

## Rename all columns

In [ ]:
orderedcols = {'Student First Name':'fname', 'Student Last Name':'lname', 'Gender':'gender','Student Email':'email', 
               'Student Phone':'phonenum','School':'schoolid','Shirt Size':'shirt','Short Size':'short',
               'Candy':'favcandy','Guardian 1 First Name':'guard1fname','Guardian 1 Last Name':'guard1lname',
               'Guardian 1 Phone':'guard1phonenum','Guardian 1 Email':'guard1email','Guardian 1 Occupation':'guard1occ',
               'Guardian 1 Shirt Size':'guard1shirt','Guardian 2 First Name':'guard2fname','Guardian 2 Last Name':'guard2lname',
               'Guardian 2 Phone':'guard2phonenum','Guardian 2 Email':'guard2email','Guardian 2 Occupation':'guard2occ',
               'Guardian 2 Shirt Size':'guard2shirt'}
def renamealldf(df):
    df.rename(orderedcols, axis=1, inplace=True)
    return df
renamedalldf= renamealldf(alldf)
renamedalldf.head()

## Add in Data from Medical Sheet

In [ ]:
cls2021[6] = cls2021[6].rename(columns ={'School District': 'schoolid', 'Last Name':'lname'})
cls2021[6] = cleancolumns(cls2021[6])
cls2021[6] = cls2021[6].astype(str)
cls2021[6] = cls2021[6].applymap(lambda x: re.sub(r'^-$', str(np.NaN), x))
cls2021[6]['comments'] =str(np.nan)
algcheck = cls2021[6][cls2021[6]["Allergies"] != str(np.NaN)].index.tolist()
for i in algcheck:
    cls2021[6].iloc[i, :]["comments"] = "Allergies: " + cls2021[6].iloc[i, :]['Allergies']

medcheck = cls2021[6][cls2021[6]["Medical Conditions"] != str(np.NaN)].index.tolist()
for i in medcheck:
    x = cls2021[6].iloc[i, :]
    if x["comments"] != str(np.NaN):
        x["comments"] = x["comments"] + '; Medical Conditions: ' + x["Medical Conditions"]
    else:
        x["comments"] = 'Medical Conditions: ' + x['Medical Conditions'] 
    cls2021[6].iloc[i, :] = x
    
drugcheck = cls2021[6][cls2021[6]["Medicines"] != str(np.NaN)].index.tolist()
for i in drugcheck:
    x = cls2021[6].iloc[i, :]
    if x["comments"] != str(np.NaN):
        x["comments"] = x["comments"] + '; Medicines: ' + x["Medicines"]
    else:
        x["comments"] = 'Medicines: ' + x['Medicines'] 
    cls2021[6].iloc[i, :] = x
    
notecheck = cls2021[6][cls2021[6]["Notes"] != str(np.NaN)].index.tolist()
for i in notecheck:
    x = cls2021[6].iloc[i, :]
    if x["comments"] != str(np.NaN):
        x["comments"] = x["comments"]+ '; Notes: ' + x['Notes'] 
    else:
        x["comments"] = 'Notes: ' + x["Notes"]
    cls2021[6].iloc[i, :] = x
del cls2021[6]["Allergies"] 
del cls2021[6]["Medical Conditions"]   
del cls2021[6]["Medicines"] 
del cls2021[6]["Notes"]   
cls2021[6]

In [ ]:
combined = pd.merge(renamedalldf, cls2021[6], on=['lname','schoolid'], how='left')
del combined['First Name']
#multiple cleo and emma 
combined =combined.drop([44,45])
combined = combined.reset_index(drop=True)
combined

## Clean all Data

In [ ]:
def transform(dforig):
    df = dforig.copy()
    
    df = df.astype(str)
    df = df.applymap(lambda x: re.sub(r'^-$', str(np.NaN), x))
    #make all lower email
    df['email'] = df['email'].str.lower()
    df['guard1email'] = df['guard1email'].str.lower()
    df['guard2email'] = df['guard2email'].str.lower()
    #remove all nonnumbers in phonenumbers
    df['phonenum'] = df['phonenum'].str.replace('-', '', regex=True).str.replace(' ', '', regex=True).str.replace('FBMessenger','', regex=True).str.replace('kevinsingh422@yahoo.com','', regex=True)
    df['guard1phonenum'] = df['guard1phonenum'].str.replace('-', '').str.replace(' ', '')
    df['guard2phonenum'] = df['guard2phonenum'].str.replace('-', '').str.replace(' ', '')
    # convert all phonenumbers to string
    df['phonenum'] = df['phonenum'].astype(str)    
    df['guard1phonenum'] = df['guard1phonenum'].astype(str)
    df['guard2phonenum'] = df['guard2phonenum'].astype(str)
    # remove all questions in sizes
    df['shirt'] = df['shirt'].str.replace('2X', 'XXL', regex=True).str.replace('3X', 'XXL', regex=True).str.replace('(', '', regex=True).str.replace(')', '', regex=True).str.replace(' ', '')
    df['short'] = df['short'].str.replace('2X', 'XXL', regex=True).str.replace('3X', 'XXL', regex=True).str.replace('(', '', regex=True).str.replace(')', '', regex=True).str.replace(' ', '')
    df['guard1shirt'] = df['guard1shirt'].str.replace('2XL', 'XXL', regex=True).str.replace('2X', 'XXL', regex=True,case=False).str.replace('3XL', 'XXL', regex=True).str.replace('3X', 'XXL', regex=True, case=False).str.replace('(', '', regex=True).str.replace(')', '', regex=True).str.replace(' ', '').str.replace('XXXL','XL').str.replace('1XL','XL')
    df['guard2shirt'] = df['guard2shirt'].str.replace('2XL', 'XXL', regex=True).str.replace('2X', 'XXL', regex=True).str.replace('3XL', 'XXL', regex=True).str.replace('3X', 'XXL', regex=True).str.replace('(', '', regex=True).str.replace(')', '', regex=True).str.replace(' ', '').str.replace('4XL','XXL')
    df['gradyear'] = 2020
    
    emcheck = df[df["Emergency Contact"] != str(np.NaN)].index.tolist()
    df['emergcontact']=np.nan
    print(emcheck)
    for i in emcheck:
        x = df.iloc[i, :]
        x['emergcontact'] = x["Emergency Contact"]+ ' ' + x["EC Phone"]
        df.iloc[i, :] =x
    del df["Emergency Contact"]
    del df["EC Phone"]
    
    relcheck = df[df["Guardian 1 Relationship"] != str(np.NaN)].index.tolist()
    print(relcheck)
    for i in relcheck:
        x = df.iloc[i, :]
        if x["comments"] != str(np.NaN):
            x['comments'] = x["comments"] + '; Guardian 1 Relationship: ' + x["Guardian 1 Relationship"]
        else:
            x['comments'] = 'Guardian 1 Relationship: ' + x["Guardian 1 Relationship"]
        df.iloc[i, :] =x
    del df["Guardian 1 Relationship"]
        
    rel2check = df[df["Guardian 2 Relationship"] != str(np.NaN)].index.tolist()
    for i in rel2check:
        x = df.iloc[i, :]
        if x["comments"] != str(np.NaN):
            x['comments'] = x["comments"] + '; Guardian 2 Relationship: ' + x["Guardian 2 Relationship"]
        else:
            x['comments'] = 'Guardian 2 Relationship: ' + x["Guardian 2 Relationship"]
        df.iloc[i, :] =x
    del df["Guardian 2 Relationship"]
    
    medcheck = df[df["Medical Conditions"] != str(np.NaN)].index.tolist()
    for i in medcheck:
        x = df.iloc[i, :]
        if x["comments"] != str(np.NaN):
            x["comments"] = x["comments"] + '; Medical Conditions: ' + x["Medical Conditions"]
        else:
            x["comments"] = 'Medical Conditions: ' + x['Medical Conditions'] 
        df.iloc[i, :] = x
    del df["Medical Conditions"]
    return df
cleancombined = transform(combined)

def intphone(df, var):
    for i in df.index:
        if df.loc[i,var] == str(np.nan):
            df.loc[i,var]== np.nan
        else:
            df.loc[i,var] = '+1'+df.loc[i,var]
intphone(cleancombined, 'phonenum')
intphone(cleancombined, 'guard1phonenum')
intphone(cleancombined, 'guard2phonenum')
cleancombined

## Write to csv

In [ ]:
#needed blank columns like transdf2021 to use same upload code
cleancombined['dob']= np.nan
cleancombined['student_ig']= np.nan
cleancombined['address']= np.nan
cleancombined['race']= np.nan
cleancombined['guard1occ']= np.nan
cleancombined['guard2occ']= np.nan

filepath = 'attollo/basic/management/commands/cls2021.csv'
cleancombined.to_csv(filepath)

In [ ]:
cleancombined["schoolid"].unique().tolist()

In [ ]:
filepath = 'attollo/basic/management/commands/cls2021.csv'
out = pd.read_csv(filepath)
out.iloc[83,:]